# Imports

# Constants

# Classes

# Python Multiprocessing

## Running Tasks in Parallel

## Process Pools and Queues

## Parallel Corpus Preprocessing

# Cluster Computing with Spark

## Anatomy of a Spark Job

## Distributing the Corpus

## RDD Operations

## NLP with Spark

### From Scikit-Learn to MLLib

### Feature extraction

### Text clustering with MLLib

### Text classification with MLLib

### Local fit, global evaluation